# Extract short term features
This script extracted the short term features from the audio files. Since each clip has a different length, we thought of padding the shorter audios to have a uniform lenght.

This idea has a couple of problems, mainly the fact that we're increasing the dimension of the dataset by adding just empty information. We never came back to this idea, but we keep it for reference.

In [1]:
from pyAudioAnalysis import ShortTermFeatures as aF
from pyAudioAnalysis import audioBasicIO as aIO 
import numpy as np 
import plotly.graph_objs as go 
import plotly
import IPython

import os
import time

import matplotlib.pyplot as plt

files = os.listdir('../Data/Train')

In [2]:
def display_time(seconds):
    if seconds <= 60:
        print('Duration: %f (s)' %seconds)
    elif 60 < seconds <= 3600:
        print('Duration: %f (min)' %(seconds/60))
    else:
        print('Duration: %f (h)' %(seconds/3600))

Load files and pad them to the same length

In [4]:
time_start = time.time()

# Load audio files and calculate the longest length
Fs = []
Signals = []
max_length = 0
for file in files:
    fs, s = aIO.read_audio_file('../../AudioWAV/%s' %file)
    Fs.append(fs)
    Signals.append(s)
    max_length = max(max_length, len(s))

# Pad all files so that they all have the same length
for i in range(len(files)):
    s = Signals[i]
    
    before = 0
    after  = max_length - len(s)
    s_pad = np.pad(s, (before,after), mode='constant', constant_values=(0,0))
    
    Signals[i] = s_pad
Signals = np.array(Signals)

time_end = time.time()
display_time(time_end-time_start)

print('Number of files: %i' %(len(files)))

Duration: 4.402534 (s)
Number of files: 7442


Extract features from all audio clips

In [ ]:
time_start = time.time()

# Extract features from the first clip to get sizes
win, step = 0.05, 0.025
s = Signals[0,:]
fs = Fs[0]
[f, fn] = aF.feature_extraction(s, fs, int(fs * win), int(fs * step))

Features = np.zeros((len(files), f.shape[0], f.shape[1]))
Features[0,:,:] = f

for i in range(1, len(files)):
    s = Signals[i,:]
    fs = Fs[i]
    [f, fn] = aF.feature_extraction(s, fs, int(fs * win), int(fs * step))
    
    Features[i,:,:] = f

time_end = time.time()
display_time(time_end-time_start)

In [14]:
np.savez('Signals', Signals=Signals, Fs=Fs)
np.savez('Features', win=win, step=step, Features=Features, fn=fn)

In [15]:
Features.shape

(7442, 68, 200)